# Sextupole - Comparing to Bmad

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2
from pytao import Tao

import impact.z as IZ
from impact.tests.z.conftest import bmad_files
from impact.z.interfaces.bmad import export_particles as tao_export_particles

In [ ]:
energy = 10e6
pz = np.sqrt(energy**2 - mec2**2)
P0 = single_particle(x=1e-3, pz=pz)

In [ ]:
!cat $bmad_files/sextupole.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "sextupole.bmad", plot="mpl")

In [ ]:
P0.write("sextupole_initial_particles.h5")
for cmd in [
    "set beam_init position_file = sextupole_initial_particles.h5",
    f"set beam_init n_particle = {len(P0)}",
    f"set beam_init bunch_charge = {P0.charge}",
    "set beam_init saved_at = *",
    "set global track_type = single",
    "set global track_type = beam",
]:
    res = tao.cmd(cmd)
    if res:
        print("\n".join(res))

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
I = IZ.ImpactZ.from_tao(
    tao, use_temp_dir=False, workdir="./sextupole-bmad", initial_particles=P0
)

In [ ]:
I.input

In [ ]:
I.input.multipole

In [ ]:
output = I.run()

In [ ]:
I.output

In [ ]:
Ptao = tao_export_particles(tao, "END")

In [ ]:
mc2 = I.input.reference_particle_mass
stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y
energy = stats.mean_energy

x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
s_tao = tao.bunch_comb("s")
p_tao = (1 + tao.bunch_comb("pz")) * tao.bunch_comb("p0c")
energy_tao = np.hypot(p_tao, mc2)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(8, 8), sharex=True)
ax1.plot(z, x, label="Impact-Z")
ax1.plot(s_tao, x_tao, "--", label="Tao")
ax1.set_ylabel(r"$x$ (m)")
ax1.legend()

ax2.plot(z, y, label="Impact-Z")
ax2.plot(s_tao, y_tao, "--", label="Tao")
ax2.set_ylabel(r"$y$ (m)")

ax3.plot(z, energy / 1e6, label="Impact-Z")
ax3.plot(s_tao, energy_tao / 1e6, "--", label="Tao")
ax3.set_ylabel(r"$E$ (MeV)")
ax3.set_xlabel(r"$s$ (m)")

I.input.plot(ax=ax4, include_marker_labels=False);